## Carbon Monitoring Project

In [ ]:
import holoviews as hv
import pandas as pd
hv.extension('bokeh')

This notebook aims to visualize the data used in the carbon monitoring project [nee_data_fusion](https://github.com/greyNearing/nee_data_fusion/) using Python tools.

The goals of this notebook:

* examine the measurements from each site
* generate some visualization or global model to predict one site from every other site.
* generate and explain model idea

To run this notebook, you will need to symlink the `data` directory of the [nee_data_fusion](https://github.com/greyNearing/nee_data_fusion/) to `flux_data` in the `examples` directory of `EarthML`. In addition, you will need `RSIF_2007_2016_05N_01L.mat` in the `examples` directory which you can download from https://gentinelab.eee.columbia.edu/content/datasets

### Loading FluxNet data ``extract_fluxnet.m``

[FluxNet](http://fluxnet.fluxdata.org/) is a worldwide collection of sensor stations that record a number of local variables relating to atmospheric conditions, solar flux and soil moisture. The data is the [nee_data_fusion](https://github.com/greyNearing/nee_data_fusion/) repository is expressed as a collection of CSV files where the site names are expressed in the filenames.

This cell defines functions to

* read in the data from all sites
* do some data munging (i.e., date parsing, `NaN` replacement)

In [ ]:
import numpy as np
import datetime
import os

In [ ]:
DAILIES_DIR = '../../nee_data_fusion/data/in_situ/fluxnet_daily/'
METADATA_CSV = '../../nee_data_fusion/data/in_situ/extracted/allflux_metadata.txt'
# DAILIES_DIR = 'flux_data/dailies/'
# METADATA_CSV = 'allflux_metadata.txt'

sites = [fname.split('_')[1] for fname in os.listdir(DAILIES_DIR)]
metadata = pd.read_csv(METADATA_CSV, header=None, names=['site', 'lat', 'lon', 'igbp', 'network'], 
                       usecols=['site', 'lat', 'lon', 'igbp'], index_col='site')

# get all the igbp codes for these sites
igbp_codes = metadata.loc[sites].igbp.unique()

In [ ]:
# Any missing metadata?
metadata.loc[sites].isnull().values.any()

In [ ]:
def _parse_days(integer):
    """ `integer` date as `20180704` to represent July 4th, 2018"""
    x = str(integer)
    d = {'year': int(x[:4]), 'month': int(x[4:6]), 'day': int(x[6:])}
    day_of_year = datetime.datetime(d['year'], d['month'], d['day']).timetuple().tm_yday
    return day_of_year

def clean(df, timestamp_col="TIMESTAMP", site='', keep=[], drop=[], predict=''):
    """
    Clean the dataset
    
    * Replace NaN's and any number less than -9990 with 0s
    * drop columns specified in `drop`
    * pull out prediction and feature matrices
    * Parse timestamp and pull out day of year ("DOY")
    """
    limit = -9990
    for i in range(50):
        df = df.replace(limit - i, np.nan)
    
    to_drop = [col for col in drop if col in df.columns]
    df.drop(columns=to_drop, inplace=True)
    for col in keep:
        if col not in df.columns:
            if 'SWC_F' in col or 'TS_F' in col:
                df[col] = 0
    
    df = df.fillna(0)
    df['DOY'] = df['TIMESTAMP'].apply(_parse_days)  
    df.pop('TIMESTAMP')
    X = df[keep]
    y = df[predict]
    return X, y

def load_fluxnet_site(site, one=False):
    """
    The main function to load data
    
    Parameters
    ----------
    site : str
        e.g., "US-CA1"
    one : bool, optional
        Whether to preform a dirty hack and create "one" dataframe that
        includes the prediction variable in the feature matrix.
        
    Returns
    -------
    X : pd.DataFrame
        Feature matrix. If ``one``, this will include the prediction variable
        and be the only thing returned.
    y : pd.DataFrame
        The prediction variable. Not returned if ``one``.
    """
    #dataRaw(dataRaw <= -9990) = 0/0 (is NaN?)
    #NaN -> zero
    prefix = 'FLX_{site}_FLUXNET'.format(site=site)
    filenames = [fname for fname in os.listdir(DAILIES_DIR)
                if fname.startswith(prefix)]
    if len(filenames) != 1:
        raise FileNotFoundError
    filename = filenames[0]
    path = '{directory}{filename}'.format(directory=DAILIES_DIR, filename=filename)
    
    raw_daily = pd.read_csv(path)    
    
    keep =  ['P_ERA',
             'TA_ERA',
             'PA_ERA',
             'SW_IN_ERA',
             'LW_IN_ERA',
             'WS_ERA',
             'SWC_F_MDS_1', 'SWC_F_MDS_2', 'SWC_F_MDS_3',
             'TS_F_MDS_1', 'TS_F_MDS_2', 'TS_F_MDS_3',
             'VPD_ERA',
             'DOY']
    drop = ["GPP_DT_VUT_USTAR50",
            "GPP_DT_CUT_USTAR50",
            "LE_F_MDS",
            "H_F_MDS"]
    predict = ["NEE_CUT_USTAR50",
               "NEE_VUT_USTAR50"]
    
    X, y = clean(raw_daily, keep=keep, drop=drop, predict=predict[0])
    X['site'] = site  # some metadata
    X['y'] = y
    return X

## Setup Dask
Dask is required to read in the CSVs and do preprocessing *quickly*.

In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd
from distributed import Client

client = Client()
client

## Read in data

In [ ]:
futures = client.map(load_fluxnet_site, sites, one=True)

In [ ]:
succeeded = [f for f in futures if not f.exception()]
failed = [f for f in futures if f.exception()]

In [ ]:
dfs = client.gather(succeeded)

## Merge data

Once the data are loaded in, they need to be joined with the metadata relating to each site.

In [ ]:
df = pd.concat(dfs)

# create a little dataframe for mapping categorical variables
a = np.zeros((len(igbp_codes), len(igbp_codes)), int)
np.fill_diagonal(a, 1)
categorical_igbp_mapper = pd.DataFrame(index=igbp_codes, columns=igbp_codes, data=a)
categorical_igbp_mapper.rename_axis('igbp', inplace=True)

# add metadata to the big dataframe
df = pd.merge(df, metadata, on='site')
df = pd.merge(df, categorical_igbp_mapper, on='igbp')

print("df shape =", df.values.shape)
if float('nan') in df.columns:
    print('nan in df.columns, removing')
    df.pop(float('nan'))

In [ ]:
# set this to False to not inlude vegetation type in the calculation
include_veg = True

show = df.sample(frac=0.10)
sites = pd.Categorical(show['site']).codes
dropped = {}
for col in ['DOY', 'site', 'lon', 'lat', 'igbp']:
    dropped[col] = show[col].copy()
    show.pop(col)
    
if not include_veg:
    for col in igbp_codes:
        dropped[col] = show[col].copy()
        show.pop(col)
        
print("{} observations and {} variables".format(*show.shape))
print("Generating a prediction with these variables: \n  {}".format(
    "\n  ".join(list(
        show.columns
    ))
))

These variables are sufficient to create the linear models at every site. However, the site information is hidden from the visualization algoritm.

* Good sanity checks:
    - lattitude encoded some structure, longitude does not

## Visualization

Linear models work well *at one site* but this is confounded by

* lat/lon
* day of year
* environment type

We want to generate some visualization that accounts for these 4 variables and helps generate some understanding.

That is, these observations lie on some manifold. We want to learn the structure of that manifold, and visualize each observation on that manifold.

This will have the finding similar observations that have a similar structure between the indepedent variables (e.g., `P_ERA`) and depedent variables (the carbon flux measurement `y`).

UMAP is a tool for this, and has firm mathematical grounding (plus, it's nice to use).

In [ ]:
import umap
reduct = umap.UMAP(verbose=True, n_epochs=None)#, n_neighbors=30)

In [ ]:
reduct.fit(show.values)

In [ ]:
embedding = reduct.embedding_
embedding

In [ ]:
cols = ['lat', 'lon', 'igbp']
s = pd.DataFrame(dropped)
s['x0'] = embedding[:, 0]
s['x1'] = embedding[:, 1]
for col in cols:
    if col in show:
        s[col] = show[col]
    else:
        assert col in s

In [ ]:
from bokeh.models import Select
from bokeh.layouts import row, widgetbox
from bokeh.palettes import Category20
from bokeh.plotting import curdoc
from holoviews.ipython.display_hooks import display
import colorcet as cc

colors = ['lat', 'lon', 'DOY', 'site', 'igbp']

def create_figure(color='lat', **kwargs):
    opts = {'plot': {'color_index': color, 'show_legend': False,
                     'width': 600, 'height': 600, 'colorbar': True,
                     'tools': ['hover']},
            'style': {'cmap': 'magma', 'legend': False}
}
    if color == 'DOY':
        opts['style']['cmap'] = cc.cm['cyclic_mrybm_35_75_c68']
    if color == 'igbp':
        opts['style']['cmap'] = 'Category20'
        opts['plot']['legend_position'] ='right'
        opts['plot']['show_legend'] = True
    if color == 'site':
        opts['style']['cmap'] = 'Category20'
        opts['plot']['colorbar'] = False
        opts['plot']['width'] = 700

    opts.update(**kwargs)
    chart = hv.Scatter(
        s, kdims=['x0', 'x1'], vdims=[color, 'site'], extents=(-15,-15,15,15)
    ).opts(plot=opts['plot'], style=opts['style'])
    return display(chart)

from ipywidgets import interactive

w = interactive(create_figure, color=colors)
w

## Taking a closer look at vegetation

In [ ]:
igbp_vegetation = {
    'ENF': '01 - Evergreen Needleleaf forest',
    'EBF': '02 - Evergreen Broadleaf forest',
    'DNF': '03 - Deciduous Needleleaf forest',
    'DBF': '04 - Deciduous Broadleaf forest',
    'MF': '05 - Mixed forest',
    'CSH': '06 - Closed shrublands',
    'OSH': '07 - Open shrublands',
    'WSA': '08 - Woody savannas',
    'SAV': '09 - Savannas',
    'GRA': '10 - Grasslands',
    'WET': '11 - Permanent wetlands',
    'CRO': '12 - Croplands',
}

In [ ]:
s['vegetation'] = s['igbp'].apply(lambda x: igbp_vegetation[x])

In [ ]:
ds = hv.Dataset(s, ['x0', 'vegetation'], ['x1', 'site'])
grouped = ds.to(hv.Scatter, kdims=['x0', 'x1'], extents=(-15,-15,15,15), vdims=['site'])

In [ ]:
# https://lpdaac.usgs.gov/about/news_archive/modisterra_land_cover_types_yearly_l3_global_005deg_cmg_mod12c1
lpdaac_palette = [
    '#008000', '#00FF00', '#99CC00', '#99FF99', '#339966', '#993366',
    '#FFCC99', '#CCFFCC', '#FFCC00', '#FF9900', '#006699', '#FFFF00'
]

In [ ]:
%%opts Scatter [width=800, height=600] (color=Cycle(lpdaac_palette), size=1, muted_alpha=0)
grouped.overlay('vegetation').options(legend_position='right')

Isolate each vegetation type and color by site id so that any site ecentricities are made clear

In [ ]:
grouped.options(color_index='site', cmap='Category20', show_legend=False, size=1, alpha=0.8).layout().cols(3)

## Prediction
Linear models work well *at one site* but this is confounded by

* lat/lon
* day of year
* environment type

In [ ]:
from sklearn.preprocessing import StandardScaler

assert 'site' not in show.columns
y = show['y'].values
X = pd.DataFrame({col: show[col].values 
                  for col in show.columns 
                  if col != 'y'})
print(X.shape)
assert 'y' not in X.columns

# tranform data matrix so 0 mean, unit variance for each feature
X = StandardScaler().fit_transform(X.values)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression

def predict(X_train, X_test, y_train):
    # find the nearest neighbors
    neighbors = NearestNeighbors(n_neighbors=50)
    neighbors.fit(X_train)
    idx = neighbors.kneighbors(X_test, return_distance=False)

    # train and fit a linear model of those values
    y_hat = []
    for point, neighbors in zip(X_test, idx):
        _X_train = X_train[neighbors]
        _y_train = y_train[neighbors]
    
        model = LinearRegression()
        model.fit(_X_train, _y_train)
        y_hat += [model.predict(point.reshape(1, -1))]
    return np.array(y_hat).flat[:]

* **TODO:** Integrate with Dask (likely wrwapping predict calls with `dask.delayed`)
* **TODO:** Select better method of finding neighbors
    * smart way of figuring out neighbors? Probably all observations within a certain radius
    * use the embedding (only if radius method doesn't work, requires reading UMAP paper)

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
sep = LeaveOneGroupOut()
corrs = []
n_splits = sep.get_n_splits(X, y, sites)
for i, (train_index, test_index) in enumerate(sep.split(X, y, sites)):
    if i % 10 == 0 and i > 0:
        print('{:0.0f}% complete'.format(100 * i / n_splits))
    y_hat = predict(X[train_index],
                    X[test_index],
                    y[train_index])
    corrs += [np.corrcoef(y_hat, y[test_index])[0, 1]]

In [ ]:
corrs = np.array(corrs)
frequencies, edges = np.histogram(corrs, 20)

c1 = hv.Histogram((frequencies, edges))
c2 = hv.VLine(np.mean(corrs), label='mode')
c1 * c2

## End of modifications
`----------------------------------------------------`


The rest of this notebook is copy-pasted from `Carbon-FLux.ipynb`. I haven't modified it yet.

`----------------------------------------------------`

### Adding RSIF data ``collocate_data_types.m``

RSIF is the 'Reconstructed Solar Induced Fluorescence' expressing solar energy flux (power) per meter squared arriving on the Earth's surface derived from a vegetation signal.

In [ ]:
import scipy.io
rsif = scipy.io.loadmat('RSIF_2007_2016_05N_01L.mat')


The goal now is to add the RSIF time series added at positions of stations, then normalize the time dimension (resample to make the data sets have the same temporal sampling) and finally to perform a linear regression analysis on the combined data.

Note that the matlab file reference above was downloaded from https://gentinelab.eee.columbia.edu/content/datasets

In [ ]:
%%opts Image [width=700 height=500 clipping_colors={'NaN': 'gray'}] Points (marker='x' color='cyan')

mdata = rsif['RSIF'] # NaNs outside of land area.
def rsif_image(day):
    return hv.Image(mdata[:,:,day], kdims=['lon', 'lat'], vdims=['RSIF'], bounds=(-180,-90,180,90))

rsif_dmap = hv.DynamicMap(rsif_image, kdims=['day']).redim.values(day=range(mdata.shape[2]))
raw_site_positions = {site:site_lat_lon(metadata, site) for site in sites}
site_positions = {site:(lon,lat) for (site, (lat,lon)) in raw_site_positions.items() if None not in (lat,lon)}
rsif_dmap * hv.Points(site_positions.values())

* Algorithm might be useful: Smoothing?
* Mismatched temporospatial satellite imagery.

1. Timestamp per lat/lon. Comes with triples. 
2. Polar: e.g 1pm local time.
3. Global (processed product?)

https://science.nasa.gov/earth-science/earth-science-data/data-processing-levels-for-eosdis-data-products

1. Level 0: Raw data (direct sensor flux). Highest resolution. Minimal model - good for ML.
2. Level 1: Sensor geometry.
3. Level 2: spatial regridding  
4. Level 3: check: physical variable. Primary science product.
5. Level 4: check: model added value.



## Sampling the RSIF signal at the sites

In [ ]:
tables = []
for day in range(mdata.shape[2]):
    image = rsif_dmap.select(day=day)
    table = image.sample(samples=site_positions.values())
    tables.append(table.add_dimension('site', 0, site_positions.keys()))

In [ ]:
tabulated = hv.HoloMap({day:tables[day] for day in range(mdata.shape[2])}, kdims=['Day'])
tabulated

## Viewing the RSIF time series per site

In [ ]:
site_RSIF = tabulated.table().to(hv.Curve, 'Day', 'RSIF').drop_dimension(['lat', 'lon'])
site_RSIF

### biweekly_averaging.m

'RSIF' is put onto the same time sampling as the fluxnet data, adding an 'RSIF' field to the fluxnet dataframe for each site.

### main_lr_fluxnet.m


Finding the informative variables.

Main step:

```
model{s} = stepwiselm(X,y,''constant'',''Criterion'',''aic'',''Upper'',''linear'')
```

Using [stepwiselm](https://uk.mathworks.com/help/stats/stepwiselm.html):

```
mdl = stepwiselm(X,y,modelspec) creates a linear model of the responses y to the predictor variables in the data matrix X, using stepwise regression to add or remove predictors. modelspec is the starting model for the stepwise procedure.
```

Where ``allData`` is the table (effectively the dataframe):

```
cols= [3:11,14:22];
X = allData(:,cols,s);
y = allData(:,12,s);
```

* 3:11 : 'LAT','LON','P_ERA','TA_ERA','PA_ERA','SW_IN_ERA','LW_IN_ERA','WS_ERA','LE_F_MDS'
* 12: 'H_F_MDS' (Sensible heat flux, gapfilled using MDS method)

Check. NEE is the desired predicted. Column 13.

### main_ann_fluxnet.m

Main step for size ``s`` in ``trainANN`` using [``train``](https://uk.mathworks.com/help/nnet/ref/train.html) in the neural network toolbox:

```
% net = fitnet(parms.nodes,parms.trainFcn);
net = feedforwardnet(parms.nodes,parms.trainFcn);
[net,~] = train(net,x,t);
```

```
Xdex= [2:10,15,18,21,24];
Ydex = 13;
```

